# RW - API management functions

These functions will cover the functions need to manage assets within the RW-API and will be base on the code develop for [OW migration from staging to db](https://github.com/resource-watch/notebooks/blob/develop/ResourceWatch/example_migrate_script.ipynb)

In [2]:
%run './pydantic_data_classes_rw.ipynb'

In [12]:
from requests_toolbelt import sessions
from requests_toolbelt.auth.handler import AuthHandler
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from datetime import datetime

import types
import json
import logging
import getpass

from pydantic import parse_obj_as

logger = logging.getLogger()
logger.setLevel(logging.INFO)

print('\n'.join(f'{module.__name__}=={getattr(module, "__version__", None)}' for  i, module in globals().items() if isinstance(module, types.ModuleType)))

builtins==None
builtins==None
requests_toolbelt.sessions==None
types==None
json==2.0.9
logging==0.5.1.2
getpass==None


In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [3]:
DEFAULT_TIMEOUT = 60 # seconds

class TimeoutHTTPAdapter(HTTPAdapter):
    '''
    
    '''
    def __init__(self, *args, **kwargs):
        self.timeout = DEFAULT_TIMEOUT
        if "timeout" in kwargs:
            self.timeout = kwargs["timeout"]
            del kwargs["timeout"]
        super().__init__(*args, **kwargs)

    def send(self, request, **kwargs):
        timeout = kwargs.get("timeout")
        if timeout is None:
            kwargs["timeout"] = self.timeout
        return super().send(request, **kwargs)

In [4]:
def auth(session):
    '''
    Authenticates into the RW API
    '''
    print(f'You are login into {bcolors.HEADER}{bcolors.BOLD}RW api{bcolors.ENDC}')
    
    headers = {'Content-Type': 'application/json'}
    payload = json.dumps({ 'email': f'{input(f"Email: ")}',
                           'password': f'{getpass.getpass(prompt="Password: ")}'})
    response = session.post('/auth/login',  headers = headers,  data = payload)
    
    print(f'{bcolors.OKGREEN}Successfully logged into RW api{bcolors.ENDC}')
    
    return response.json().get('data').get('token')

In [5]:
def rwApiSession(serverUrl="https://api.resourcewatch.org"):
    '''
    Creates a sesion object for making calls to the RW api.
    '''
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    
    assert_status_hook = lambda response, *args, **kwargs: response.raise_for_status()

    httpsSession = sessions.BaseUrlSession(base_url = serverUrl )
    
    httpsSession.mount("https://", TimeoutHTTPAdapter(max_retries=retries))
    httpsSession.hooks["response"] = [assert_status_hook]
    
    token = auth(httpsSession)
    
    httpsSession.headers['Authorization'] = f'Bearer {token}'
    
    return httpsSession

In [6]:
def copyAssetBody(asset, excludeList=['createdAt', 'updatedAt','clonedHost', 'errorMessage', 'taskId', 
                                      'status', 'sources','userId', 'slug', 'dataset', 'layer', 
                                      'widget', 'metadata', 'vocabulary']):
    '''
    Copy a body dict to a new dict excluding some keys or not defined values.
    '''
    response = {}
    response.update(asset)
    
    for key, value in asset.items():
        if (key in excludeList or value is None or (type(value) == dict and len(value) == 0) ):
            response.pop(key, None)
    
    if 'provider' in response.keys() and response['provider'] =='cartodb':
        response.pop('tableName', None)
    
    return response

In [7]:
def getAssetList(session, application: List='rw', env='production', includes='layer'):
    '''
    Gets a list of assets from the selected env or from the constrained dataset list
    For our needs we will ignore other includer than layer.
    '''
    url = '/v1/dataset'
    payload={
        'env':'production',
        'application':'rw',
        'status':'saved',
        'includes':'layer',
        'page[size]':100
    }
    response = []
    data = session.get(url, params=payload).json()
    
    response.extend(data['data'])
    
    # If pagination is needed we will access the data chunk by chunk
    if data['meta']['total-pages'] > 1:
        for n in range(2, data['meta']['total-pages'] + 1):
            payload['page[number]'] = n
            response.extend(session.get(url, params=payload).json()['data'])
    
    return parse_obj_as(List[Dataset], response)

In [1]:
def getLayerList(session):
    '''
    Gets a list of assets from the selected env or from the layers.
    '''
    url = '/v1/layer'
    payload={
        'env':'production',
        'application':'rw',
        'status':'saved',
        'page[size]':100
    }
    response = []
    data = session.get(url, params=payload).json()
    
    response.extend(data['data'])
    
    # If pagination is needed we will access the data chunk by chunk
    if data['meta']['total-pages'] > 1:
        for n in range(2, data['meta']['total-pages'] + 1):
            payload['page[number]'] = n
            response.extend(session.get(url, params=payload).json()['data'])
    return parse_obj_as(List[Layer], response)

In [9]:
def backupAssets(session, env = 'prod'):
    '''
    save a backup of production data just in case we need to recreate it again
    '''
    data = getAssetList(session)

    with open(f'RW_{env}_backup_{datetime.now().strftime("%Y%m%d-%H%M%S")}.json', 'w') as outfile:
        json.dump(data, outfile)

In [10]:
def createOrUpdatedataset(session, dataset, toEnv = 'staging', destinationDatasetId = None):
    '''
    Copy the dataset from one env to the other
    '''
    if dataset.get('type')!='dataset':
        return None
    
    baseDatasetUrl = f'/v1/dataset'

    body = {'dataset': copyAssetBody(dataset.get('attributes'))}
    
    logger.debug(body)
    
    # Upsert operation
    if destinationDatasetId: 
        return session.patch(f'{baseDatasetUrl}/{destinationDatasetId}',json = body['dataset']).json()
    else:
        return session.post(baseDatasetUrl, body['dataset']).json()

In [11]:
def createOrUpdateLayer(session, datasetId, layer, toEnv = 'staging', destinationLayerId = None):
    '''
    Copy the layer from one env to the other
    '''
    if layer.get('type')!='layer':
        return None
    baseDatasetUrl = f'/v1/dataset/{datasetId}/layer'

    body = copyAssetBody(layer.get('attributes'))
    
    logger.debug(body)
    
    # Upsert operation
    if destinationLayerId: 
        return session.patch(f'{baseDatasetUrl}/{destinationLayerId}', json = body['dataset']).json()
    else:
        return session.post(baseDatasetUrl, body['dataset']).json()